# Plot the astrometric signatures of known exoplanet host stars 

In this notebook we query two exoplanet catalogs and estimate the expected amplitude (semi-major axis) of the host star's Keplerian motion caused by the orbiting planet.

Dependencies:
    astropy, pandas, astroquery, pystrometry

In [ ]:
import glob
import os
import pprint 

import astropy.constants as constants
from astropy.table import Table, vstack
import astropy.units as u
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astroquery.exoplanet_orbit_database import ExoplanetOrbitDatabase
from IPython.display import display
import matplotlib as mp
import matplotlib.pylab as pl
import pandas as pd

from pystrometry.pystrometry import semimajor_axis_barycentre_angular

In [ ]:
LOCAL_PATH = os.path.dirname(os.getcwd())
TEMPORARY_DIR = os.path.join(LOCAL_PATH, 'temp')
if os.path.isdir(TEMPORARY_DIR) is False:
    os.makedirs(TEMPORARY_DIR)
print(LOCAL_PATH)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)     
%matplotlib inline
mp.rcParams['figure.figsize'] = (18, 9)
mp.rcParams['font.size'] = 20
mp.rcParams['ytick.direction'] = 'in'
mp.rcParams['xtick.direction'] = 'in'

In [ ]:
save_plot = True

### Some useful functions

https://exoplanetarchive.ipac.caltech.edu/docs/program_interfaces.html

In [ ]:
def get_exoplanets(selected_table_name, overwrite=False):
    
    local_data_file_name = {'eod': 'exoplanet_orbit_database_table.ecsv',
                           'nasa': 'nasa_exoplanet_archive_table.ecsv'}

    eod_file = os.path.join(TEMPORARY_DIR, local_data_file_name[selected_table_name])

    if not os.path.isfile(eod_file) or overwrite:
        if selected_table_name == 'eod':
            table = ExoplanetOrbitDatabase.get_table()
        elif selected_table_name == 'nasa':
            # https://exoplanetarchive.ipac.caltech.edu/docs/program_interfaces.html
            table = NasaExoplanetArchive.query_criteria(table="exoplanets", select="*", cache=True)
            #         table = NasaExoplanetArchive.query_criteria(table="cumulative", select="*", cache=True)
            #         table_kepler = NasaExoplanetArchive.query_criteria(table="cumulative", select="*", cache=True)

        table.write(eod_file, overwrite=True)

    table = Table.read(eod_file)
    return table


parameter_mapping_all = {'eod': {'period': 'PER',
                     'ecc': 'ECC',
                     'm2': 'MASS',
                     'omega': 'OM',
                     'plx': 'PAR',
                     'star_mass': 'MSTAR',
                     'distance_pc': 'DIST'
                     },
                     'nasa':{'period': 'pl_orbper',         # https://exoplanetarchive.ipac.caltech.edu/docs/API_exoplanet_columns.html
                             'm2': 'pl_bmassj',
                             'plx': 'gaia_plx',
                             'star_mass': 'st_mass',
                             'distance_pc': 'st_dist'
                             }}


def set_astrometric_observables(table, selected_table_name):
    """Compute photocentre orbit size and other parameters."""

    parameter_mapping = parameter_mapping_all[selected_table_name]                     

    planet_mass_mj = table[parameter_mapping['m2']]
    parallax_mas = table[parameter_mapping['plx']]
    period_day = table[parameter_mapping['period']]
    star_mass_msun = table[parameter_mapping['star_mass']]

    table['a_phot_mas'] = semimajor_axis_barycentre_angular(star_mass_msun, planet_mass_mj, period_day, parallax_mas)
    table['period_year'] = (table[parameter_mapping['period']].to(u.year))
    table['a_phot_muas'] = table['a_phot_mas']*1000

    table['log10_distance_pc'] = np.log10(table[parameter_mapping['distance_pc']])
    
    return table

def set_a_phot_mas(table, selected_table_name):
    """Compute photocentre orbit size (works on pandas dataframe)."""

    parameter_mapping = parameter_mapping_all[selected_table_name]                     

    planet_mass_mj = table[parameter_mapping['m2']]
    parallax_mas = table[parameter_mapping['plx']]
    period_day = table[parameter_mapping['period']]
    star_mass_msun = table[parameter_mapping['star_mass']]

    table['a_phot_mas'] = semimajor_axis_barycentre_angular(star_mass_msun, planet_mass_mj, period_day, parallax_mas)
    table['a_phot_muas'] = table['a_phot_mas']*1000
    
    return table

### Get table from NasaExoplanetArchive

In [ ]:
selected_table_name = 'nasa'  
table = get_exoplanets(selected_table_name) 
table = set_astrometric_observables(table, selected_table_name)

### Get table from ExoplanetOrbitDatabase for KOIs

In [ ]:
eod_table = get_exoplanets('eod')  
eod_table = set_astrometric_observables(eod_table, 'eod')

eod_df = eod_table.to_pandas()
eod_df_koi = eod_df[eod_df['EANAME'].str.contains('Kepler').fillna(False)]

## Solar system planets  at 10 pc     
http://www.windows2universe.org/our_solar_system/planets_table.html

In [ ]:
solar_system = pd.DataFrame()

solar_system['period_year'] = [0.24 ,0.62, 1, 1.88, 11.86, 29.46, 84.01, 164.8]
solar_system['period_day'] = solar_system['period_year'] * u.year.to(u.day)
solar_system['planet_mass_mj'] = np.array([0.055,0.815, 1, 0.107, 318, 95, 15, 17]) * u.M_earth.to(u.M_jup)
solar_system['distance_pc'] = 10
solar_system['parallax_mas'] = 1e3/solar_system['distance_pc']
solar_system['planet_name'] = ['Me','V','E','Ma','J','S','U','N']
solar_system['star_mass_msun'] = 1

solar_system['a_phot_mas'] = semimajor_axis_barycentre_angular(solar_system['star_mass_msun'], solar_system['planet_mass_mj'], solar_system['period_day'], solar_system['parallax_mas'])
solar_system['a_phot_muas'] = solar_system['a_phot_mas']*1e3

display(solar_system)

## Individual systems (mostly Gaia-HIP accelerations)

In [ ]:
individuals = pd.DataFrame()

individuals['planet_name'] = ['HD 33632 Ab', 'Proxima c', 'HD 113337 c', 'HD 38529 c', 'pi Men b', 'beta Pic b', 'eps indi A b']
individuals['ref'] = ['Currie+20', 'Kervella+20', 'Xuan+20', 'Xuan+20', 'De Rosa+20', 'Brandt+20', 'Feng+19']

individuals['period_year'] = np.array([91* u.year.to(u.day), 1900, 3165, 2135, 2089.11, 8880, 45.2*u.year.to(u.day)])* u.day.to(u.year)
individuals['period_day'] = individuals['period_year'] * u.year.to(u.day)
individuals['planet_mass_mj'] = np.array([46, 12* u.M_earth.to(u.M_jup), 14, 18, 13.01, 9.8, 3.25]) 
individuals['distance_pc'] = [26.56, 1000./768.5, 1000./27.64, 1000./23.611, 1000/ 54.7052, 1000./50.58, 1000./274.8]
individuals['parallax_mas'] = 1e3/individuals['distance_pc']
individuals['star_mass_msun'] = [1.108, 0.122, 1.40, 1.36, 1.094, 1.83, 0.754]

individuals['a_phot_mas'] = semimajor_axis_barycentre_angular(individuals['star_mass_msun'], individuals['planet_mass_mj'], individuals['period_day'], individuals['parallax_mas'])
individuals['a_phot_muas'] = individuals['a_phot_mas']*1e3

display(individuals)

In [ ]:
print(individuals[['planet_name', 'ref']].to_csv(index=False, sep=';'))

## Systems with measured photocentre orbits

In [ ]:
df = table.to_pandas()

# ups And c,d
selected_systems = df[df['hip_name'].str.contains('HIP 7513').fillna(False)].iloc[[1,2]]
selected_systems['pl_bmassj'] = [13.98, 10.25]
selected_systems['ref'] = 'McArthur+10'

#  HD 38529 c
individual_system = df[df['pl_name'].str.contains('HD 38529 c').fillna(False)]
individual_system['pl_bmassj'] = [17.6]
individual_system['ref'] = 'Benedict+10'
selected_systems = pd.concat([selected_systems, individual_system])

#  HD 128311 b
individual_system = df[df['pl_name'].str.contains('HD 128311 b').fillna(False)]
individual_system['pl_bmassj'] = [3.789]
individual_system['ref'] = 'McArthur+14'
selected_systems = pd.concat([selected_systems, individual_system])

#  HD 202206 c
individual_system = df[df['pl_name'].str.contains('HD 202206 c').fillna(False)]
individual_system['ref'] = 'Benedict+17'
selected_systems = pd.concat([selected_systems, individual_system])

#  eps Eri b (Mawet+17 ignores the HST astrometry)
# individual_system = df[df['pl_name'].str.contains('eps Eri b').fillna(False)]
# individual_system['pl_bmassj'] = [1.55]
# individual_system['ref'] = 'Benedict+06'
# selected_systems = pd.concat([selected_systems, individual_system])

#  GJ 676A b
individual_system = df[df['pl_name'].str.contains('GJ 676 A b').fillna(False)]
individual_system['pl_bmassj'] = [6.7]
individual_system['ref'] = 'Sahlmann+16'
selected_systems = pd.concat([selected_systems, individual_system])

selected_systems = set_a_phot_mas(selected_systems, 'nasa')

individual_system = pd.DataFrame()
individual_system['pl_name'] = ['TVLM 513–46546 b']
individual_system['pl_bmassj'] = [0.38]
individual_system['a_phot_mas'] = [0.128]
individual_system['ref'] = ['Curiel+20']
individual_system['period_year'] = [220./u.year.to(u.day)]
individual_system['distance_pc'] = [1000./93.405]

individual_system['a_phot_muas'] = individual_system['a_phot_mas']*1000
individual_system['st_dist'] = individual_system['distance_pc']

selected_systems = pd.concat([selected_systems, individual_system])
display(selected_systems[['pl_name', 'period_year', 'pl_bmassj', 'a_phot_muas', 'ref', 'pl_def_refname']])

In [ ]:
#  df[df['hip_name'].str.contains('HIP 85647').fillna(False)].iloc[0]

In [ ]:
print(selected_systems[['pl_name', 'ref']].to_csv(index=False, sep=','))

In [ ]:
# np.array(df.columns)

## Generate figure

In [ ]:
include_koi = True
include_individuals = True
include_orbits = True


def annotate_df(row):  
    #     https://stackoverflow.com/questions/15910019/annotate-data-points-while-plotting-from-pandas-dataframe
    ax.annotate(row.planet_name, (row.period_year, row.a_phot_muas),
                xytext=(10,-5), 
                textcoords='offset points',
                size=18, 
                color='darkslategrey')
    

x = 'period_year' 
y = 'a_phot_muas'

if y == 'a_phot_muas':
    factor = 1000

colour_by = 'distance_pc'
norm = mp.colors.LogNorm()
colormap='viridis'
colormap='cubehelix'


fig, ax = pl.subplots()
df.plot(x, y, kind='scatter', logx=True, logy=True, c=parameter_mapping_all['nasa'][colour_by], colormap=colormap, s=30, alpha=0.9, norm=norm, ax=ax, zorder=1, label='Exoplanets')
if include_koi:
    eod_df_koi.plot(x, y, kind='scatter', logx=True, logy=True, c=parameter_mapping_all['eod'][colour_by], colormap=colormap, s=25, marker='s', alpha=0.9, zorder=-50, norm=norm, ax=ax, colorbar=False, label='KOIs')
pl.ylim((1e-8*factor, 40*factor))
pl.xlim((1e-3, 300))
solar_system.plot(x, y, kind='scatter', logx=True, logy=True, s=200, c=colour_by, ax=ax, norm=norm, colormap=colormap, colorbar=False, marker='^', label='Solar system at 10pc', edgecolor='k')
_ = solar_system.apply(annotate_df, axis=1)
if include_orbits:
    selected_systems.plot(x, y, kind='scatter', logx=True, logy=True, s=300, c=parameter_mapping_all['nasa'][colour_by], ax=ax, norm=norm, colormap=colormap, colorbar=False, label='Measured orbits',  marker='*', edgecolor='k') #,    
if include_individuals:
    individuals.plot(x, y, kind='scatter', logx=True, logy=True, s=200, c=colour_by, ax=ax, norm=norm, colormap=colormap, colorbar=False, label='Hip-Gaia systems',  marker='D', edgecolor='k') #,
pl.xlabel('Orbital period (year)')
pl.ylabel('Minimum semimajor axis (micro-arcsec)')
ax.grid(ls='--', color='0.8')
ax.set_zorder(-52)

xticks = [0.001, 0.01, 0.1, 1, 10, 100] 
xtick_labels = np.array(['0.001', '0.01', '0.1', '1', '10', '100'])
pl.xticks(xticks, xtick_labels)

yticks = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 1e4] 
ytick_labels = np.array(yticks).astype(str)
ytick_labels = np.array(['0.0001', '0.001', '0.01', '0.1', '1', '10', '100', '1000', '10000'])
pl.yticks(yticks, ytick_labels)

cax = fig.get_axes()[1]
cax.set_ylabel('Distance (pc)')
pl.legend(loc=2, framealpha=0.5)

pl.text(0.68, 0.03, 'NasaExoplanetArchive & ExoplanetOrbitDatabase', horizontalalignment='left', verticalalignment='top',
        transform=pl.gca().transAxes, fontsize=10)

pl.title('Astrometric signatures of exoplanets')
if save_plot:
    figure_file_name = os.path.join(TEMPORARY_DIR, '{}_{}_{}_astrometry_signatures.png'.format(selected_table_name, include_individuals, include_orbits))
    pl.savefig(figure_file_name, transparent=False, bbox_inches='tight', pad_inches=0.05)        
display(figure_file_name)    